Источники: 

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [21]:
# импорты библиотек
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
from torch.optim import lr_scheduler

from torch.autograd import Variable
import torch.onnx

import os


In [22]:
# задаем текущую директорию
dir = os.path.abspath(os.curdir)
# в данном случае папка с изображениями лежит в папке "data"
data_dir=os.path.join(dir, "data\\")

In [23]:
# нормализация изображений и уменьшение до 32*32 пикселя
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

In [24]:
# функция преобразования изображений в тензоры
def get_dataset(data_dir, data_transforms ):
    # для тренировочного и тестового датасета
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                 shuffle=True, num_workers=4)
                  for x in ['train', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    # получаем названия классов на примере тренировочного датасета
    classes = image_datasets['train'].classes

    return dataloaders["train"], dataloaders['test'], classes, dataset_sizes

In [25]:
trainloader, testloader, classes, dataset_sizes=get_dataset(data_dir,data_transforms)
print('Classes: ',  classes)
print('The datasest have: ',  dataset_sizes ," images")

Classes:  ['cocacola', 'pepsi']
The datasest have:  {'train': 340, 'test': 60}  images


In [26]:
# задаем параметры сверточной нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [32]:
# задаем количество эпох обучения
n_epocn = 15 
# задаем критерии оптимизации и завершения обучения
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cpu")
# обучение модели на тренировочном датасете
for epoch in range(n_epocn):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # вывод статистики на печать
        running_loss += loss.item()
        if i % 15 == 14:    # print every 15 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 15))
            running_loss = 0.0
print('Finished Training')

[1,    15] loss: 0.102
[1,    30] loss: 0.312
[1,    45] loss: 0.177
[1,    60] loss: 0.367
[1,    75] loss: 0.413
[2,    15] loss: 0.262
[2,    30] loss: 0.206
[2,    45] loss: 0.132
[2,    60] loss: 0.185
[2,    75] loss: 0.227
[3,    15] loss: 0.297
[3,    30] loss: 0.233
[3,    45] loss: 0.124
[3,    60] loss: 0.264
[3,    75] loss: 0.183
[4,    15] loss: 0.418
[4,    30] loss: 0.323
[4,    45] loss: 0.240
[4,    60] loss: 0.359
[4,    75] loss: 0.214
[5,    15] loss: 0.289
[5,    30] loss: 0.308
[5,    45] loss: 0.242
[5,    60] loss: 0.212
[5,    75] loss: 0.215
[6,    15] loss: 0.243
[6,    30] loss: 0.265
[6,    45] loss: 0.166
[6,    60] loss: 0.217
[6,    75] loss: 0.271
[7,    15] loss: 0.196
[7,    30] loss: 0.180
[7,    45] loss: 0.165
[7,    60] loss: 0.169
[7,    75] loss: 0.211
[8,    15] loss: 0.279
[8,    30] loss: 0.255
[8,    45] loss: 0.205
[8,    60] loss: 0.133
[8,    75] loss: 0.142
[9,    15] loss: 0.264
[9,    30] loss: 0.131
[9,    45] loss: 0.072
[9,    60] 

In [33]:
# проверяем на тестовом датасете, как работает модель
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for printdata in list(zip(predicted,labels,outputs)):
            printclass =[classes[int(printdata[0])],classes[int(printdata[1])]]
            print('Predict class - {0}, real class - {1}'.format( printclass[0],printclass[1]))
            
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the', dataset_sizes['test'], 'test images: %d %%' % (
    100 * correct / total))

Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - cocacola, real class - cocacola
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - cocacola, real class - cocacola
Predict class - pepsi, real class - pepsi
Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - pepsi
Predict class - pepsi, real class - cocacola
Predict class - cocacola, real class - cocacola
Predict class - cocacola, real class - pepsi
Predict cl

Вывод: accuracy на достаточно высоком уровне, значит, можно использовать модель для проверки изображения в реальном времени с веб-камеры

In [34]:
# сохраняем модель
PATH =os.path.join(dir, "pytorch_model.onnx")
dummy_input = Variable(torch.randn(1, 3, 32, 32))
torch.onnx.export(net, dummy_input, PATH)